In [10]:
import numpy as np
import pandas as pd
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import ttest_rel
import pingouin as pg
from constants import (ACCURACIES_PATH, ACC_UNSHIFTED_PATH)
from utils.utils_statistical_analysis import stats_preprocessing

# Importing data and formatting it
Imports all the accuracies and brings them into a table that can be used for statistical analysis. Also segments the data to analyse only certain blocks.

In [11]:
accuracies_data = pd.read_csv(ACCURACIES_PATH, header=None)
stats_data = stats_preprocessing(accuracies_data)

#stats_data_pre = stats_data.loc[(stats_data['Run'] == 1) | (stats_data['Run'] == 2)]
#stats_data_treatment = stats_data.loc[(stats_data['Run'] == 3) | (stats_data['Run'] == 4)]
#stats_data_post = stats_data.loc[(stats_data['Run'] == 5) | (stats_data['Run'] == 6)]

#stats_data.info()

# Output
Exporting data to use in other programs

In [12]:
# np.savetxt("data/stats_data.csv", stats_data, delimiter=",")
# np.savetxt("data/stats_data_treatment.csv", stats_data_treatment, delimiter=",")
# np.savetxt("data/stats_data_pre.csv", stats_data_pre, delimiter=",")
# np.savetxt("data/stats_data_post.csv", stats_data_post, delimiter=",")

# ANOVA
calculates within-subjects ANOVA

In [13]:
model_1 = AnovaRM(data=stats_data, depvar='Accuracy', subject='Subject', within=['Run', 'Treatment']).fit()
print("\033[4m" + "Model 1" + "\033[0m")
print(model_1)
print("")

model_2 = pg.rm_anova(dv='Accuracy', within=['Run', 'Treatment'], subject='Subject', data=stats_data)
print("\033[4m" + "Model 2" + "\033[0m")
print(model_2)
print("")

Model 1
                   Anova
              F Value Num DF  Den DF  Pr > F
--------------------------------------------
Run            0.7133 5.0000 155.0000 0.6144
Treatment      0.3744 1.0000  31.0000 0.5451
Run:Treatment  0.9344 5.0000 155.0000 0.4605


Model 2
            Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              Run  0.025461      5    155  0.005092  0.713259  0.614357   
1        Treatment  0.003380      1     31  0.003380  0.374367  0.545095   
2  Run * Treatment  0.045883      5    155  0.009177  0.934356  0.460456   

   p-GG-corr       ng2       eps  
0   0.599474  0.005755  0.895642  
1   0.545095  0.000768  1.000000  
2   0.447861  0.010324  0.818152  



# T-Test
Calculates a t-test comparing individual runs to each other

In [14]:
sham_4 = stats_data.query('Treatment == 1 and Run == 6')['Accuracy']
stim_4 = stats_data.query('Treatment == 2 and Run == 6')['Accuracy']
print(np.mean(sham_4))
print(np.mean(stim_4))
print("")

t_test_1 = pg.ttest(sham_4, stim_4, paired=True)
print("\033[4m" + "T-Test 1" + "\033[0m")
print(t_test_1)

t_test_2 = ttest_rel(sham_4, stim_4, alternative='less')
print("\033[4m" + "T-Test 2" + "\033[0m")
print(t_test_2)

0.6212327075098814
0.6298326333992095

T-Test 1
              T  dof alternative     p-val          CI95%   cohen-d   BF10  \
T-test -0.32962   31   two-sided  0.743904  [-0.06, 0.04]  0.080766  0.199   

           power  
T-test  0.072758  
T-Test 2
Ttest_relResult(statistic=-0.32961955725514863, pvalue=0.37195219941705226)


# Performance Split
Splitting dataset into high and low performers and calculating ANOVA

In [15]:
subject_performance = stats_data.Accuracy.to_numpy()
subject_performance = np.mean(subject_performance.reshape(-1, 12), axis=1)
performance_index = np.argpartition(subject_performance, int(len(subject_performance)/2))
performance_index = performance_index + 1
performance_index = np.array_split(performance_index,2)

low_performers = stats_data[~(stats_data.Subject.isin(performance_index[1]))]
high_performers = stats_data[~(stats_data.Subject.isin(performance_index[0]))]

anova_low_performers = pg.rm_anova(dv='Accuracy', within=['Run', 'Treatment'], subject='Subject', data=low_performers)
anova_high_performers = pg.rm_anova(dv='Accuracy', within=['Run', 'Treatment'], subject='Subject', data=high_performers)

print("\033[4m" + "Low Performers" + "\033[0m")
print(np.mean(low_performers.Accuracy))
print(anova_low_performers)
print("")
print("\033[4m" + "High Performers" + "\033[0m")
print(np.mean(high_performers.Accuracy))
print(anova_high_performers)

Low Performers
0.5751168272397891
            Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              Run  0.052037      5     75  0.010407  1.544898  0.186236   
1        Treatment  0.001240      1     15  0.001240  0.122413  0.731296   
2  Run * Treatment  0.019499      5     75  0.003900  0.322364  0.898046   

   p-GG-corr       ng2       eps  
0   0.208021  0.028173  0.704322  
1   0.731296  0.000690  1.000000  
2   0.846930  0.010746  0.734675  

High Performers
0.6768251468489238
            Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              Run  0.011203      5     75  0.002241  0.298189  0.912426   
1        Treatment  0.013792      1     15  0.013792  1.778969  0.202176   
2  Run * Treatment  0.066757      5     75  0.013351  1.742729  0.135251   

   p-GG-corr       ng2       eps  
0   0.878480  0.007314  0.802277  
1   0.202176  0.008989  1.000000  
2   0.158664  0.042056  0.733412  


# Variance Split
Splitting dataset into subjects with high and low variance and calculating ANOVA

In [24]:
variances = accuracies_data.var(axis=1)
variance_index = np.argpartition(variances, int(len(variances)/2))
variance_index = variance_index + 1
variance_index = np.array_split(variance_index,2)

low_variance = stats_data[~(stats_data.Subject.isin(variance_index[0]))]
high_variance = stats_data[~(stats_data.Subject.isin(variance_index[1]))]

anova_low_variance = pg.rm_anova(dv='Accuracy', within=['Run', 'Treatment'], subject='Subject', data=low_variance)
anova_high_variance = pg.rm_anova(dv='Accuracy', within=['Run', 'Treatment'], subject='Subject', data=high_variance)

print("\033[4m" + "Low Variance" + "\033[0m")
print("Mean Accuracy = " + str(np.mean(low_variance.Accuracy)))
print("Mean Variance = " + str(np.mean(variances[variance_index[0] - 1])))
print(anova_low_variance)
print("")
print("\033[4m" + "High Variance" + "\033[0m")
print("Mean Accuracy = " + str(np.mean(high_variance.Accuracy)))
print("Mean Variance = " + str(np.mean(variances[variance_index[1] - 1])))
print(anova_high_variance)

Low Variance
Mean Accuracy = 0.6141853315766358
Mean Variance = 0.00584928075254753
            Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              Run  0.032505      5     75  0.006501  0.806424  0.548686   
1        Treatment  0.000055      1     15  0.000055  0.004575  0.946969   
2  Run * Treatment  0.099513      5     75  0.019903  1.441028  0.219511   

   p-GG-corr       ng2       eps  
0   0.520948  0.013376  0.761385  
1   0.946969  0.000023  1.000000  
2   0.235965  0.039851  0.727375  

High Variance
Mean Accuracy = 0.6377566425120772
Mean Variance = 0.01110257718679758
            Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              Run  0.031132      5     75  0.006226  1.006857  0.419712   
1        Treatment  0.005593      1     15  0.005593  0.872648  0.365018   
2  Run * Treatment  0.005062      5     75  0.001012  0.177526  0.970228   

   p-GG-corr       ng2       eps  
0   0.407353  0.016564  0.730165  
1   0.3650

# Comparing day 1 to day 2
Using the unshifted data to test if there is a difference in the performance between day 1 and day 2 and to see if there is a learning effect

In [17]:
accuracies_unshifted_data = pd.read_csv(ACC_UNSHIFTED_PATH, header=None)
stats_data_unshifted = stats_preprocessing(accuracies_unshifted_data)

stats_data_day1 = stats_data.loc[(stats_data['Treatment'] == 1)]
stats_data_day2 = stats_data.loc[(stats_data['Treatment'] == 2)]

unshifted_anova = pg.rm_anova(dv='Accuracy', within=['Run', 'Treatment'], subject='Subject', data=stats_data_unshifted)
anova_day1 = pg.rm_anova(dv='Accuracy', within=['Run'], subject='Subject', data=stats_data_day1)
anova_day2 = pg.rm_anova(dv='Accuracy', within=['Run'], subject='Subject', data=stats_data_day2)

print("\033[4m" + "ANOVA comparing day 1 and 2" + "\033[0m")
print(unshifted_anova)
print("")
print("\033[4m" + "ANOVA of day 1" + "\033[0m")
print(anova_day1)
print("")
print("\033[4m" + "ANOVA of day 2" + "\033[0m")
print(anova_day2)

ANOVA comparing day 1 and 2
            Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              Run  0.025461      5    155  0.005092  0.713259  0.614357   
1        Treatment  0.000086      1     31  0.000086  0.009457  0.923155   
2  Run * Treatment  0.037047      5    155  0.007409  0.750078  0.587216   

   p-GG-corr       ng2       eps  
0   0.599474  0.005739  0.895642  
1   0.923155  0.000020  1.000000  
2   0.560758  0.008330  0.806473  

ANOVA of day 1
  Source  ddof1  ddof2         F    p-unc       ng2       eps
0    Run      5    155  0.183656  0.96843  0.003376  0.822987

ANOVA of day 2
  Source  ddof1  ddof2         F     p-unc       ng2       eps
0    Run      5    155  1.465109  0.204336  0.028667  0.897004


# Comparing Baseline to Stimulation Phase
Calculating the ANOVA comparing baseline trials with trial 3 and 4. First on both days, then only on the day of the actual stimulation.

In [18]:
stats_data_short = stats_data.loc[(stats_data['Run'] == 1) | (stats_data['Run'] == 2) | (stats_data['Run'] == 3) | (stats_data['Run'] == 4)]
stats_data_short.Run = stats_data_short.Run.replace(2, 1)
anova_short = pg.rm_anova(dv='Accuracy', within=['Run', 'Treatment'], subject='Subject', data=stats_data_short)

stats_data_short_treatment = stats_data_short.loc[(stats_data_short['Treatment'] == 2)]
anova_short_treatment = pg.rm_anova(dv='Accuracy', within=['Run'], subject='Subject', data=stats_data_short_treatment)

tukey = pairwise_tukeyhsd(endog=stats_data_short_treatment['Accuracy'],
                          groups=stats_data_short_treatment['Run'],
                          alpha=0.05)

print("\033[4m" + "ANOVA comparing Baseline to run 3 and 4" + "\033[0m")
print(anova_short)
print("")
print("\033[4m" + "ANOVA comparing Baseline to run 3 and 4 in treatment condition" + "\033[0m")
print(anova_short_treatment)
print("")
print("\033[4m" + "Pairwise comparisons" + "\033[0m")
print(tukey)

/var/folders/h8/2kfl3g3j6qb3s84n6r41gc8m0000gn/T/ipykernel_33321/684600618.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats_data_short.Run = stats_data_short.Run.replace(2, 1)


ANOVA comparing Baseline to run 3 and 4
            Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              Run  0.019604      2     62  0.009802  1.875857  0.161817   
1        Treatment  0.006703      1     31  0.006703  0.747643  0.393865   
2  Run * Treatment  0.034661      2     62  0.017331  2.250362  0.113900   

   p-GG-corr       ng2       eps  
0   0.163623  0.010137  0.960784  
1   0.393865  0.003490  1.000000  
2   0.117997  0.017785  0.931006  

ANOVA comparing Baseline to run 3 and 4 in treatment condition
  Source  ddof1  ddof2         F     p-unc       ng2       eps
0    Run      2     62  3.244521  0.045697  0.051021  0.894647

Pairwise comparisons
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     1      3  -0.0126   0.85 -0.0674 0.0423  False
     1      4   0.0405   0.19 -0.0143 0.0954  False
     3      4   0.0531 0.1193 -0.0102 0.116